In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds 

In [3]:
tf.__version__

'2.12.0'

In [4]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
import numpy as np

X = np.arange(1, 101, step=0.1)
y = [x + 10 for x in X]

X = tf.cast(tf.constant(X), dtype=tf.float32)
y = tf.cast(tf.constant(y), dtype=tf.float32)

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, input_shape=(1,), activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(
    loss=tf.keras.losses.mean_absolute_error,
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
    metrics=['mean_absolute_error']
)

model.fit(X, y, epochs=100)

Epoch 1/100


2023-04-09 22:09:22.880045: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


32/32 [==============================] - 2s 14ms/step - loss: 13.4682 - mean_absolute_error: 13.4682
Epoch 2/100
32/32 [==============================] - 0s 11ms/step - loss: 4.6444 - mean_absolute_error: 4.6444
Epoch 3/100
32/32 [==============================] - 0s 12ms/step - loss: 8.4674 - mean_absolute_error: 8.4674
Epoch 4/100
32/32 [==============================] - 0s 11ms/step - loss: 8.4158 - mean_absolute_error: 8.4158
Epoch 5/100
32/32 [==============================] - 0s 11ms/step - loss: 4.7097 - mean_absolute_error: 4.7097
Epoch 6/100
32/32 [==============================] - 0s 11ms/step - loss: 7.3260 - mean_absolute_error: 7.3260
Epoch 7/100
32/32 [==============================] - 0s 11ms/step - loss: 3.2205 - mean_absolute_error: 3.2205
Epoch 8/100
32/32 [==============================] - 0s 12ms/step - loss: 5.0367 - mean_absolute_error: 5.0367
Epoch 9/100
32/32 [==============================] - 0s 11ms/step - loss: 7.4772 - mean_absolute_error: 7.4772
Epoch 10/10

In [8]:
model.predict([10, 20, 30])

1/1 [==============================] - 0s 136ms/step


array([[20.463522],
       [30.684338],
       [40.892048]], dtype=float32)

In [9]:
%%time

import tensorflow_datasets as tfds

print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.list_physical_devices('GPU')

(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

batch_size = 128

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(batch_size)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(batch_size)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, kernel_size=(3, 3),
                 activation='relu'),
  tf.keras.layers.Conv2D(64, kernel_size=(3, 3),
                 activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
#   tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
#   tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)

model.fit(
    ds_train,
    epochs=12,
    validation_data=ds_test,
)

TensorFlow version: 2.12.0
Num GPUs Available:  1
Cause: Unable to locate the source code of <function normalize_img at 0x106d9cca0>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function normalize_img at 0x106d9cca0>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function normalize_img at 0x106d9cca0>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Epoch 1/12
469/469 [==============================] - 13s 22ms/step - loss: 0.1672 - accuracy: 0.9510 - val_loss: 0.0537 - val_accuracy: 0.9829
Epoch 2/12
469/469 [==============================] - 10s 21ms/step - loss: 0.0444 - accuracy: 0.9865 - val_loss: 0.0461 - val_accuracy: 0.9848
Epoch 3/12
469/469 [==============================] - 9s 20ms/step - loss: 0.0291 - accuracy: 0.9909 - val_loss: 0.0411 - val_accuracy: 0.9866
Epoch 4/12
469/469 [==============================